In [8]:
from KeRLas import defaultQModel, Brain, TimedGymEnv, Trainer
from KeRLas.models import DirectDiffModel, DQN
from KeRLas.policies import BoltzmannQPolicy
import numpy as np
import gym, math
from cartpole import CartPoleEnv
from cartpole_swing import CartPoleSwingEnv

#env = GymEnv(CartPoleEnv(), tlimit=300)
env = TimedGymEnv(CartPoleEnv(), tlimit=300)

space = env.observation_space
high = np.array(
    [
        2.4,
        1.0,
        12 * 2 * math.pi / 360,
        1.0
    ]
)
env.RandomObservationSpace = gym.spaces.Box(-high, high)

observation_width = env.observation_space.shape[0]
nactions = env.action_space.n

rlmodel = DirectDiffModel(defaultQModel(observation_width, nactions), 0.9)
policy = BoltzmannQPolicy(1.0)
brain = Brain(rlmodel, policy)
trainer = Trainer(env, brain, 0.5, 10000)




WARN: gym.spaces.Box autodetected dtype as <type 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <type 'numpy.float32'>. Please provide explicit dtype.


In [9]:
class Callback:
    
    def onEpisodeBegin(self, env, agents, observations):
        env.render()

    def onStep(self, env, env_done, feedback):
        env.render()
        
    def onEpisodeEnd(self, env, record):
        env.render()

from KeRLas import GymPlayer

p = GymPlayer(env, brain, callback=Callback())

temps = [10.0, 1.0, 0.1, 0.01, 0.001, 0.0001]
mbsize = 50
steps_per_epoch = 100
epochs = 20


for i in xrange(20):
    training_tau = temps[i%len(temps)]
    policy.tau = training_tau
    
    trainer.train(mbsize, steps_per_epoch, epochs)
    
    nsum = 0
    N = 5
    policy.tau = 0.0001
    for _ in range(N):
        history = p.runEpisode()
        #print len(history)
        nsum += len(history)
    print training_tau, float(nsum)/N

10.0 12.4
1.0 9.8
0.1 10.4
0.01 9.6
0.001 11.4
0.0001 83.0
10.0 9.8
1.0 17.8
0.1 17.6
0.01 20.4
0.001 12.0
0.0001 12.4
10.0 83.0
1.0 12.6
0.1 37.8
0.01 17.6
0.001 56.6
0.0001 17.4
10.0 14.4
1.0 74.0


In [ ]:
for _ in xrange(100):
    p.runEpisode()

In [ ]:
def fcn(x,y,z=0, *params, **args):
    pass